In [ ]:
import pandas as _hex_pandas
import datetime as _hex_datetime
import json as _hex_json

In [ ]:
hex_scheduled = _hex_json.loads("false")

In [ ]:
hex_user_email = _hex_json.loads("\"example-user@example.com\"")

In [ ]:
hex_run_context = _hex_json.loads("\"logic\"")

In [ ]:
!pip install pydeck
import pandas as pd
import pydeck as pdk
import math
from IPython.core.display import display, HTML

In this section, we aggregate all trips in the trips table using a SQL cell. A Python cell identifies the top 10 stations with the most trips started. A Hex Chart Cell displays the count of trips from each of these stations.

In [ ]:
# import jinja2
# raw_query = """
#     select start_station_name as station_name, USERTYPE as user_type,
#     count(*) as trip_count
#     from demo.TRIPS
#     group by 1, 2
#     order by trip_count
# """
# sql_query = jinja2.Template(raw_query).render(vars())

In [ ]:
total_station_trips = station_trips_by_usertype.groupby(['STATION_NAME'])['TRIP_COUNT'].agg('sum').reset_index()
top_stations = total_station_trips.sort_values('TRIP_COUNT', ascending=False).head(10)['STATION_NAME'].to_list()
filtered_station_trips = station_trips_by_usertype[station_trips_by_usertype['STATION_NAME'].isin(top_stations)]

In [ ]:
import altair
chart_filtered_station_trips = altair.Chart.from_json("""
{
    "width": 500,
    "height": 500,
    "$schema": "https://vega.github.io/schema/vega-lite/v4.json",
    "layer": [
        {
            "data": {
                "name": "layer00"
            },
            "mark": {
                "tooltip": true,
                "type": "bar",
                "clip": true
            },
            "encoding": {
                "x": {
                    "field": "STATION_NAME",
                    "type": "nominal",
                    "sort": null
                },
                "y": {
                    "field": "TRIP_COUNT",
                    "type": "quantitative"
                },
                "color": {
                    "field": "USER_TYPE",
                    "type": "nominal"
                }
            }
        }
    ],
    "resolve": {
        "scale": {
            "x": "independent",
            "y": "independent"
        }
    },
    "datasets": {
        "layer00": [
            {
                "name": "dummy",
                "value": 0
            }
        ]
    }
}
""")
chart_filtered_station_trips.datasets.layer00 = filtered_station_trips.to_json(orient='records')
chart_filtered_station_trips.display(actions=False)

## Activity by Citi Bike Station

See the selected station's top rider destinations in the selected period.

A drop down menu, dynamically populated by the top 10 list created in the previous section, allows a user to select a station of interest along with a start and end date for their analysis. Using jinja, a SQL cell uses these inputs to filter and aggregate the trips data to the relevant set.

This output is visualized with a Hex Chart Cell to show the top 50 destination stations from the user selection in the selected time period. The second visualization is created using Python to visualize the destinations and their frequencies on a map.

In [ ]:

_hex_start_station_mapper = {__builtins__.str(v): v for v in __builtins__.list(top_stations)}
start_station = _hex_start_station_mapper[__builtins__.str(_hex_json.loads("\"Broadway & E 22 St\""))] if __builtins__.str(_hex_json.loads("\"Broadway & E 22 St\"")) in _hex_start_station_mapper and _hex_json.loads("\"Broadway & E 22 St\"") is not None else top_stations[0] if __builtins__.len(top_stations) > 0 else None
del _hex_start_station_mapper
_hex_formatters._hex_FilledDynamicValue(
    "top_stations",
    top_stations,
    max_size=2000,
    max_size_in_bytes=5242880
)

In [ ]:
start_date = _hex_datetime.datetime.strptime("01/01/2019", "%m/%d/%Y").date();

In [ ]:
end_date = _hex_datetime.datetime.strptime("12/31/2019", "%m/%d/%Y").date();

In [ ]:
# import jinja2
# raw_query = """
#     select end_station_name as end_station_name, END_STATION_LATITUDE, END_STATION_LONGITUDE,
#     count(*) as trip_count
#     from demo.TRIPS
#     where START_STATION_NAME  = {{ start_station }}
#     AND STARTTIME >= {{ start_date }} AND STARTTIME <= {{ end_date }}
#     group by 1,2,3
#     order by trip_count DESC
# """
# sql_query = jinja2.Template(raw_query).render(vars())

In [ ]:
filtered_end_station_trips = end_station_trips.head(50)

#### Top 50 destinations for trips starting at {{ start_station }}

In [ ]:
import altair
chart_filtered_end_station_trips = altair.Chart.from_json("""
{
    "width": 500,
    "height": 500,
    "$schema": "https://vega.github.io/schema/vega-lite/v4.json",
    "layer": [
        {
            "data": {
                "name": "layer00"
            },
            "mark": {
                "tooltip": true,
                "type": "bar",
                "clip": true
            },
            "encoding": {
                "x": {
                    "field": "END_STATION_NAME",
                    "type": "nominal",
                    "sort": null
                },
                "y": {
                    "field": "TRIP_COUNT",
                    "type": "quantitative"
                }
            }
        }
    ],
    "resolve": {
        "scale": {
            "x": "independent",
            "y": "independent"
        }
    },
    "datasets": {
        "layer00": [
            {
                "name": "dummy",
                "value": 0
            }
        ]
    }
}
""")
chart_filtered_end_station_trips.datasets.layer00 = filtered_end_station_trips.to_json(orient='records')
chart_filtered_end_station_trips.display(actions=False)

In [ ]:
#CONVERT STRINGS TO FLOAT
end_station_trips['END_STATION_LATITUDE'] = end_station_trips['END_STATION_LATITUDE'].astype(float)
end_station_trips['END_STATION_LONGITUDE'] = end_station_trips['END_STATION_LONGITUDE'].astype(float)
end_station_trips['TRIP_COUNT'] = end_station_trips['TRIP_COUNT'].astype(float)

#### Destinations for trips starting at {{ start_station }}

In [ ]:
# Define a layer to display on a map


layer = pdk.Layer(
    'ColumnLayer',  # `type` positional argument is here
    data=end_station_trips,
    get_position=["END_STATION_LONGITUDE","END_STATION_LATITUDE"],
    get_elevation="TRIP_COUNT",
    elevation_scale=1,
    radius=75,
    get_fill_color=["TRIP_COUNT / 8", "TRIP_COUNT/4", "TRIP_COUNT / 8", 140],
    pickable=True,
    auto_highlight=True,
)

# Set the viewport location
view_state = pdk.ViewState(
    longitude=-74, latitude=40.74, zoom=12, min_zoom=11.5, max_zoom=13, pitch=35,
)

# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html("hexagon_layer.html")

<iframe
 width="100%"
 height=500
 frameborder="0"
 srcdoc="<!DOCTYPE html>
<html>
 <head>
 <meta http-equiv="content-type" content="text/html; charset=UTF-8" />
 <title>pydeck</title>
 <script src="https://api.tiles.mapbox.com/mapbox-gl-js/v1.13.0/mapbox-gl.js"></script>
 <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap-theme.min.css" />
 <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/font-awesome/4.6.3/css/font-awesome.min.css" />
 <script src='https://cdn.jsdelivr.net/npm/@deck.gl/jupyter-widget@~8.5.*/dist/index.js'></script>
 <style>
 body {
 margin: 0;
 padding: 0;
 overflow: hidden;
}

#deck-map-container {
 width: 100%;
 height: 100%;
 background-color: black;
}

#map {
 pointer-events: none;
 height: 100%;
 width: 100%;
 position: absolute;
 z-index: 1;
}

#deckgl-overlay {
 z-index: 2;
 background: none;
}

#deck-map-wrapper {
 width: 100%;
 height: 100%;
}

#deck-container {
 width: 100vw;
 height: 100vh;
}
 </style>
 </head>
 <body>
 <div id="deck-container">
 </div>
 </body>
 <script>
 const jsonInput = {"initialViewState": {"latitude": 40.74, "longitude": -74, "maxZoom": 13, "minZoom": 11.5, "pitch": 35, "zoom": 12}, "layers": [{"@@type": "ColumnLayer", "autoHighlight": true, "data": [{"END_STATION_LATITUDE": 40.73704984, "END_STATION_LONGITUDE": -73.99009296, "END_STATION_NAME": "E 17 St & Broadway", "TRIP_COUNT": 3030.0}, {"END_STATION_LATITUDE": 40.751873, "END_STATION_LONGITUDE": -73.977706, "END_STATION_NAME": "Pershing Square North", "TRIP_COUNT": 2490.0}, {"END_STATION_LATITUDE": 40.736502, "END_STATION_LONGITUDE": -73.97809472, "END_STATION_NAME": "E 23 St & 1 Ave", "TRIP_COUNT": 2218.0}, {"END_STATION_LATITUDE": 40.73587678, "END_STATION_LONGITUDE": -73.98205027, "END_STATION_NAME": "E 20 St & 2 Ave", "TRIP_COUNT": 2045.0}, {"END_STATION_LATITUDE": 40.73454567, "END_STATION_LONGITUDE": -73.99074142, "END_STATION_NAME": "Broadway & E 14 St", "TRIP_COUNT": 1806.0}, {"END_STATION_LATITUDE": 40.749156, "END_STATION_LONGITUDE": -73.9916, "END_STATION_NAME": "W 31 St & 7 Ave", "TRIP_COUNT": 1724.0}, {"END_STATION_LATITUDE": 40.73314259, "END_STATION_LONGITUDE": -73.97573881, "END_STATION_NAME": "E 20 St & FDR Drive", "TRIP_COUNT": 1685.0}, {"END_STATION_LATITUDE": 40.74691959, "END_STATION_LONGITUDE": -74.00451887, "END_STATION_NAME": "W 22 St & 10 Ave", "TRIP_COUNT": 1638.0}, {"END_STATION_LATITUDE": 40.74173969, "END_STATION_LONGITUDE": -73.99415556, "END_STATION_NAME": "W 21 St & 6 Ave", "TRIP_COUNT": 1623.0}, {"END_STATION_LATITUDE": 40.74475148, "END_STATION_LONGITUDE": -73.99915362, "END_STATION_NAME": "W 22 St & 8 Ave", "TRIP_COUNT": 1496.0}, {"END_STATION_LATITUDE": 40.733812191966315, "END_STATION_LONGITUDE": -73.98054420948029, "END_STATION_NAME": "1 Ave & E 18 St", "TRIP_COUNT": 1406.0}, {"END_STATION_LATITUDE": 40.7403432, "END_STATION_LONGITUDE": -73.98955109, "END_STATION_NAME": "Broadway & E 22 St", "TRIP_COUNT": 1211.0}, {"END_STATION_LATITUDE": 40.7505853470215, "END_STATION_LONGITUDE": -73.9946848154068, "END_STATION_NAME": "8 Ave & W 31 St", "TRIP_COUNT": 1169.0}, {"END_STATION_LATITUDE": 40.73935542, "END_STATION_LONGITUDE": -73.99931783, "END_STATION_NAME": "W 15 St & 7 Ave", "TRIP_COUNT": 1164.0}, {"END_STATION_LATITUDE": 40.72966729392978, "END_STATION_LONGITUDE": -73.98067966103554, "END_STATION_NAME": "E 13 St & Avenue A", "TRIP_COUNT": 1162.0}, {"END_STATION_LATITUDE": 40.73221853, "END_STATION_LONGITUDE": -73.98165557, "END_STATION_NAME": "1 Ave & E 16 St", "TRIP_COUNT": 1141.0}, {"END_STATION_LATITUDE": 40.73492695, "END_STATION_LONGITUDE": -73.99200509, "END_STATION_NAME": "University Pl & E 14 St", "TRIP_COUNT": 1121.0}, {"END_STATION_LATITUDE": 40.73020660529954, "END_STATION_LONGITUDE": -73.99102628231049, "END_STATION_NAME": "Lafayette St & E 8 St", "TRIP_COUNT": 1072.0}, {"END_STATION_LATITUDE": 40.7454973, "END_STATION_LONGITUDE": -74.00197139, "END_STATION_NAME": "9 Ave & W 22 St", "TRIP_COUNT": 1043.0}, {"END_STATION_L